# Demand forecast analysis
---
- integrate multiple files into one
- fill or drop the missing observations
- identify any trends and show it in graph
---

### Import libraries and define universal variables

In [1]:
import numpy as np
import pandas as pd
import re
from time import sleep
import math
import matplotlib.pyplot as plt
import codecs

### Import data
*** please don't proceec to the next cell before data tables is shown below the cell.***

In [2]:
# file paths
size_file = "raw_data/サイズ表記.xlsx"
category_file = "raw_data/商品分類表_180410.xlsx"
color_file = "raw_data/商品カラー略号表180407.xlsx"
sale_file="raw_data/売上データ2014-2017.csv"

with codecs.open(sale_file, "r", "Shift-JIS", "ignore") as file:  # import UTF8 based csv file 
    sales_chunk = pd.read_table( file,
                                 delimiter=",",
                                 parse_dates=['sales_date'],
                                 header=0,
                                 iterator=True,
                                 chunksize=1000,
                                 na_values={'sales_proceed': 0, 'gross_profit': 0, 'area_id': 0},
                                 index_col=['product_id'],
                                 names=['sales_id', 
                                        'product_id',
                                        'product_name',
                                        'retail_price',
                                        'sales_proceeds',
                                        'gross_profit', 
                                        'number_of_sales',
                                        'sales_date',
                                        'raw_cost', 
                                        'most_recent_purchase_date',
                                        'most_recent_order_date',
                                        'number_of_inventory-effective_number',
                                        'manufacture_country_id',
                                        'manufacture_country'] )
    
    sales_df = pd.concat(sales_chunk)  # This makes all chunks saved in memory
    print(sales_df.head(4))
    
# import excel files
# for size data
size_df = pd.read_excel( size_file, sheet_name='Q_最終メモ', header=0, index_col=0 )
size_df.columns = ['size','quality','brand']
size_df.index.name = 'product_id'
print(size_df.head(10))

# for category data
category_df= pd.read_excel(category_file,  sheet_name='Sheet1', index_col=0 )
category_df.columns = ['product_name',
                       'material_type_id',
                       'material_type',
                       'manufacture_country_id',
                       'manufacture_country',
                       'category_id',
                       'category']
category_df.index.name = 'product_id'
print(category_df.head(8))

# for color data
color_df = pd.read_excel(color_file, sheet_name='商品カラー略号表180407', index_col=1).iloc[:,:2]
color_df.columns = ['english_name','japanese_name']
color_df.index.name = 'color_id'
print(color_df.head(8))

            sales_id                                    product_name  \
product_id                                                             
103591      20239746  アンティークシダー　フォールディング　　　　  テーブル　（Ｌ）　　　　　　　　　　　　　　   
102862      20239747  アンティークシダー　カードホルダー　　　　　  　　　　　　　　　　　　　　　　　　　　　　   
104782      20239747  ティーフフィル　カード　ラック　　　　　　　  　　　　　　　　　　　　　　　　　　　　　　   
999000      20239747  立替　送料　　　　　　　　　　　　　　　　　  　　　　　　　　　　　　　　　　　　　　　　   

            retail_price sales_proceeds gross_profit  number_of_sales  \
product_id                                                              
103591             18000           9900         6770                1   
102862              2200           1210          742                1   
104782              5500           3025         1986                1   
999000                 0            600          600                1   

           sales_date  raw_cost  most_recent_purchase_date  \
product_id                                                   
1035

### Integrate all dataframe into one

In [3]:
# merging all loaded data into one dataframe
integrated_df = sales_df.loc[:, ['sales_id','product_name','retail_price','sales_proceeds','gross_profit','number_of_sales','sales_date','raw_cost','manufacture_country']]
integrated_df['category'], integrated_df['material_type'] = category_df['category'], category_df['material_type']
integrated_df['size'],integrated_df['quality'],integrated_df['brand'] = size_df['size'],size_df['quality'],size_df['brand']
integrated_df.head(5)

,sales_id,product_name,retail_price,sales_proceeds,gross_profit,number_of_sales,sales_date,raw_cost,manufacture_country,category,material_type,size,quality,brand
product_id,,,,,,,,,,,,,,
103591,20239746,アンティークシダー フォールディング テーブル （Ｌ）,18000,9900,6770,1,2014-08-01,3130.16,中国 部門１,インテリア雑貨,木製品,８８×６０×７２ｃｍ,NaN,NaN
102862,20239747,アンティークシダー カードホルダー,2200,1210,742,1,2014-08-01,507.62,中国 部門１,文具・デスク用品,木製品,１８．５×２４．５×８．５ｃｍ,（ＣＥＤＡＲ）,NaN
104782,20239747,ティーフフィル カード ラック,5500,3025,1986,1,2014-08-01,1111.26,中国 部門１,実用小物・消耗品,金属製品,４２×２０×３６ｃｍ,ＩＲＯＮ,NaN
999000,20239747,立替 送料,0,600,600,1,2014-08-01,0.00,NaN,その他,その他,NaN,NaN,NaN
190038,20239751,ネックレス インターバル ヴァリエ ストーン,3800,4560,2592,2,2014-08-01,984.00,韓国,アクセサリー,金属製品,４５ｃｍ,ＳＩＬＶＥＲ９２５（１６－Ｋ ＧＯＬＤ ＰＬＡＴＥＤ） ＰＵ：ＡＭＥＴＨＹＳＴ Ｌ・ＧＹ：Ｌ...,NaN


### Extracting data from values
- extract color information from product_id 
- extract sub-category and size_letter information from product_name
- extract wholesale price from retail_price, sales_proceed, raw_cost

#### Extract color name from product_id by comparing with color_df

In [4]:


# [TODO] - extract out the chunk logics
def get_color_from_product_ids( product_ids, color_df, chunk=50000, lang='japanese_name'):
    ''' 
    Generator function
    Des: extract color information from product_ids while using color dataframe
    and return color list in a specified chunk in order to save memory 
    @product_ids: list
    @color_df: panda dataframe
    @chunk: integer
    @lang: object
    '''
    prev_chunk = 0
    next_chunk = chunk
    last_loop = False
    max_chunk = len(product_ids)
    while True: # loop through product_ids until all values are evaluated
        colors = []
        

        # if the next chunk is over the size of product_id list,
        # then re-define the next chunk with the product_id list size
        if next_chunk > max_chunk:
            last_loop = True
            next_chunk = max_chunk
            
        # the last two characters of product_id matches with color_id
        # if it doesn't match, add to color list as no color 
        for product_id in product_ids[prev_chunk:next_chunk]:
            color_id = product_id[-2:] 
            color = '色なし'
            if color_id in color_df.index:
                color = color_df.loc[color_id,lang]
            colors.append(color)
        yield colors

        if last_loop:
            print("{} color name has been extracted from product_id".format(next_chunk))
            break
        else:
            print("Extracting color name from product_id - currently extracted number: {}".format(next_chunk))
        
        # defining the next chunk
        prev_chunk = next_chunk
        next_chunk = next_chunk + chunk

    
color_lists = get_color_from_product_ids(integrated_df.index.values, color_df)
colors = []
for color_list in color_lists:
    colors = colors + color_list

# Adding extracted color series as a column
integrated_df['color'] = colors
integrated_df.head(5)

Extracting color name from product_id - currently extracted number: 50000
Extracting color name from product_id - currently extracted number: 100000
Extracting color name from product_id - currently extracted number: 150000
Extracting color name from product_id - currently extracted number: 200000
Extracting color name from product_id - currently extracted number: 250000
Extracting color name from product_id - currently extracted number: 300000
Extracting color name from product_id - currently extracted number: 350000
Extracting color name from product_id - currently extracted number: 400000
Extracting color name from product_id - currently extracted number: 450000
Extracting color name from product_id - currently extracted number: 500000
Extracting color name from product_id - currently extracted number: 550000
Extracting color name from product_id - currently extracted number: 600000
Extracting color name from product_id - currently extracted number: 650000
Extracting color name from

,sales_id,product_name,retail_price,sales_proceeds,gross_profit,number_of_sales,sales_date,raw_cost,manufacture_country,category,material_type,size,quality,brand,color
product_id,,,,,,,,,,,,,,,
103591,20239746,アンティークシダー フォールディング テーブル （Ｌ）,18000,9900,6770,1,2014-08-01,3130.16,中国 部門１,インテリア雑貨,木製品,８８×６０×７２ｃｍ,NaN,NaN,色なし
102862,20239747,アンティークシダー カードホルダー,2200,1210,742,1,2014-08-01,507.62,中国 部門１,文具・デスク用品,木製品,１８．５×２４．５×８．５ｃｍ,（ＣＥＤＡＲ）,NaN,色なし
104782,20239747,ティーフフィル カード ラック,5500,3025,1986,1,2014-08-01,1111.26,中国 部門１,実用小物・消耗品,金属製品,４２×２０×３６ｃｍ,ＩＲＯＮ,NaN,色なし
999000,20239747,立替 送料,0,600,600,1,2014-08-01,0.00,NaN,その他,その他,NaN,NaN,NaN,色なし
190038,20239751,ネックレス インターバル ヴァリエ ストーン,3800,4560,2592,2,2014-08-01,984.00,韓国,アクセサリー,金属製品,４５ｃｍ,ＳＩＬＶＥＲ９２５（１６－Ｋ ＧＯＬＤ ＰＬＡＴＥＤ） ＰＵ：ＡＭＥＴＨＹＳＴ Ｌ・ＧＹ：Ｌ...,NaN,色なし


#### Extract subcategory from product_name by comparing with subcategory_df

#### Extract material from product_name and quality by comparing with material_df
- [TODO] don't forget to get the numerical vallue after the listed material: ex) cotton 100%

#### Extract brand information from brand

#### Extract sub-category and size_letter info from product_name

In [444]:
# [TODO] - extract out the chunk logics
def get_subcategories_and_size_letter_from_name( names, chunk=50000):
    ''' 
    Generator function
    '''
    prev_chunk = 0
    next_chunk = chunk
    last_loop = False
    max_chunk = len(names)
    while True: # loop through product_ids until all values are evaluated
        subcategories = []
        size_letters = []
        

        # if the next chunk is over the size of product_id list,
        # then re-define the next chunk with the product_id list size
        if next_chunk > max_chunk:
            last_loop = True
            next_chunk = max_chunk
            
        # [TODO] logics goes here
        # - extract size letter
        # - extract sub-category and clean it
        for name in names[prev_chunk:next_chunk]:

            # extracing out size letter information
            found = re.search("\（.*?\）",name)
            if found:
                size_letter = name[found.start()+1].lower()
                name = name[:found.start()] + name[found.end():]
            else:
                size_letter = 'no size letter'
                
            # extracing subcategory information
#             subcategory = ','.join(map(lambda x:x.lower(), sorted(name.split())))
            subcategory = name.lower()
            # clean subcategory
            # - step one - unique values
            # - compare one by one
                        
            subcategories.append(subcategory)
            size_letters.append(size_letter)
        yield {'subcategory':subcategories, 'size_letter': size_letters }

        if last_loop:
            print("{} subcategory and size letter has been extracted from product_name".format(next_chunk))
            break
        else:
            print("Extracting subcategory and size letter from product_name - currently extracted number: {}".format(next_chunk))
        
        # defining the next chunk
        prev_chunk = next_chunk
        next_chunk = next_chunk + chunk

        
# getting generator list which contains extracted subcategory and size letter values
product_names = [w.replace('\u3000', ' ') for w in integrated_df['product_name'].tolist()]
extracted_lists = get_subcategories_and_size_letter_from_name(product_names)

# looping through generator list in batches to get subcategory and size_letter series 
subcategories = []
size_letters = []
for extracted in extracted_lists:
    subcategories = subcategories + extracted['subcategory']
    size_letters = size_letters + extracted['size_letter']
    
# adding extracted data to integrated dataframe as new columns
# integrated_df['sub_category'] = subcategories
integrated_df['size_letter'] = size_letters
# integrated_df.head(5)

integrated_df

Extracting subcategory and size letter from product_name - currently extracted number: 50000
Extracting subcategory and size letter from product_name - currently extracted number: 100000
Extracting subcategory and size letter from product_name - currently extracted number: 150000
Extracting subcategory and size letter from product_name - currently extracted number: 200000
Extracting subcategory and size letter from product_name - currently extracted number: 250000
Extracting subcategory and size letter from product_name - currently extracted number: 300000
Extracting subcategory and size letter from product_name - currently extracted number: 350000
Extracting subcategory and size letter from product_name - currently extracted number: 400000
Extracting subcategory and size letter from product_name - currently extracted number: 450000
Extracting subcategory and size letter from product_name - currently extracted number: 500000
Extracting subcategory and size letter from product_name - cu

,sales_id,product_name,retail_price,sales_proceeds,gross_profit,number_of_sales,sales_date,raw_cost,manufacture_country,category,material,size,quality,brand,sub_category,size_letter
product_id,,,,,,,,,,,,,,,,
103591,20239746,アンティークシダー フォールディング テーブル （Ｌ）,18000,9900,6770,1,2014-08-01,3130.16,中国 部門１,インテリア雑貨,木製品,８８×６０×７２ｃｍ,NaN,NaN,"アンティークシダー,テーブル,フォールディング",ｌ
102862,20239747,アンティークシダー カードホルダー,2200,1210,742,1,2014-08-01,507.62,中国 部門１,文具・デスク用品,木製品,１８．５×２４．５×８．５ｃｍ,（ＣＥＤＡＲ）,NaN,"アンティークシダー,カードホルダー",no size letter
104782,20239747,ティーフフィル カード ラック,5500,3025,1986,1,2014-08-01,1111.26,中国 部門１,実用小物・消耗品,金属製品,４２×２０×３６ｃｍ,ＩＲＯＮ,NaN,"カード,ティーフフィル,ラック",no size letter
999000,20239747,立替 送料,0,600,600,1,2014-08-01,0.00,NaN,その他,その他,NaN,NaN,NaN,"立替,送料",no size letter
190038,20239751,ネックレス インターバル ヴァリエ ストーン,3800,4560,2592,2,2014-08-01,984.00,韓国,アクセサリー,金属製品,４５ｃｍ,ＳＩＬＶＥＲ９２５（１６－Ｋ ＧＯＬＤ ＰＬＡＴＥＤ） ＰＵ：ＡＭＥＴＨＹＳＴ Ｌ・ＧＹ：Ｌ...,NaN,"インターバル,ストーン,ネックレス,ヴァリエ",no size letter
190039,20239751,ブレスレット インターバル ヴァリエ ストーン,3000,3600,2062,2,2014-08-01,769.00,韓国,アクセサリー,金属製品,１８．５ｃｍ,ＳＩＬＶＥＲ９２５（１６－Ｋ ＧＯＬＤ ＰＬＡＴＥＤ） ＰＵ：ＡＭＥＴＨＹＳＴ Ｌ・ＧＹ：Ｌ...,NaN,"インターバル,ストーン,ブレスレット,ヴァリエ",no size letter
190044,20239751,ブレスレット アレンジ ウォーターパール,3000,3600,2224,2,2014-08-01,688.00,韓国,アクセサリー,金属製品,１８ｃｍ,ＳＩＬＶＥＲ９２５（１６－Ｋ ＧＯＬＤ ＰＬＡＴＥＤ） ＷＡＴＥＲ ＰＥＡＲＬ ＊ポーチ付,NaN,"アレンジ,ウォーターパール,ブレスレット",no size letter
190046,20239751,ネックレス ＢＲＡＳＳ ティップ,4200,5040,3090,2,2014-08-01,975.00,韓国,アクセサリー,金属製品,５０ｃｍ,ＳＩＬＶＥＲ９２５（１６－Ｋ ＧＯＬＤ ＰＬＡＴＥＤ） ＢＲＡＳＳ＝ＰＩＰＥ,NaN,"ティップ,ネックレス,ｂｒａｓｓ",no size letter
190047,20239751,ピアス ＢＲＡＳＳ ティップ,2800,3360,2212,2,2014-08-01,574.00,韓国,アクセサリー,金属製品,１．５×４．５ｃｍ,ＳＩＬＶＥＲ９２５（１６－Ｋ ＧＯＬＤ ＰＬＡＴＥＤ） ＢＲＡＳＳ＝ＰＩＰＥ,NaN,"ティップ,ピアス,ｂｒａｓｓ",no size letter


### Wrangling with missing or bogus data or any 0 values
- Either drop NaN data or fillin with reasonable value
- clean size information
- Calculate the wholesale price

### Export integrated dataframe as a csv in dist directory

In [3]:
integrated_df.to_csv('dist/cleaned_data.csv')

NameError: name 'integrated_df' is not defined